In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics
!pip install opencv-python-headless
!pip install paddleocr
!pip install paddlepaddle

In [ ]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import Image

In [ ]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="your-api-key")
project = rf.workspace("sodtware-engineer").project("license-detector-8ygy6")
version = project.version(1)
dataset = version.download("yolov11")


In [ ]:
dataset.location

In [ ]:
!yolo task=detect mode=train data={dataset.location}/data.yaml model="yolo11n.pt" epochs=150 imgsz=640

In [ ]:
!yolo task=detect mode=predict model="/content/model.pt" conf=0.6 source={dataset.location}/test/images  save=True


In [ ]:
# Install the necessary libraries
from ultralytics import YOLO
import cv2
import os
from paddleocr import PaddleOCR

# Load the pre-trained YOLO model (replace with your new model path)
model = YOLO('/content/model.pt')

# Initialize PaddleOCR for text recognition
ocr = PaddleOCR(use_angle_cls=True, lang='en')

# Input folder containing images
input_folder = '/content/License-Detector/test/images'

# Get all image file names in the folder
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Minimum confidence threshold
confidence_threshold = 0.7

# Iterate over all image files in the input folder
for image_file in image_files:
    # Full path of the image
    input_image_path = os.path.join(input_folder, image_file)

    # Load the input image
    image = cv2.imread(input_image_path)

    # Perform detection on the image
    results = model(image)

    # Iterate over the detected objects
    detected_text = ""
    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0]  # Confidence score

            # Filter detections based on confidence
            if conf < confidence_threshold:
                continue

            cls = int(box.cls[0])  # Class ID
            label = model.names[cls]  # Class label
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates

            # Crop the bounding box area from the image
            cropped_image = image[y1:y2, x1:x2]

            # Use PaddleOCR to detect text within the cropped image
            result_text = ocr.ocr(cropped_image, cls=True)

            # Extract the detected text from the OCR result
            for line in result_text[0]:
                detected_text += line[1][0] + " "  # Append detected text

    # Print the extracted text along with the image name
    print(f"Extracted text from {image_file}:")
    print(detected_text)
    print("-" * 50)  # Separator for clarity
